##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ノートブックで TensorBoard を使用する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tensorboard/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

TensorBoard は、[Colab](https://colab.research.google.com/) や [Jupyter](https://jupyter.org/) などのノートブックエクスペリエンス内で直接使用できます。そのため、結果の共有や既存のワークフローへの TensorBoard の統合を行い、ローカルに何もインストールせずに TensorBoard を使用することができます。

## セットアップ

TF 2.0 をインストールし、TensorBoard ノートブック拡張機能を読み込んで起動します。

**Jupyter ユーザー:** Jupyter と TensorBoard を同じ virtualenv にインストールしている場合は、このまま先にお進みください。異なる Conda/virtualenv 環境に対してグローバルの Jupyter インストールとカーネルがあるといったより複雑なセットアップを使用している場合は、`tensorboard` バイナリが Jupyter ノートブックのコンテキスト内の `PATH` にあることを確認する必要があります。これには、環境の `bin` ディレクトリを `PATH` に付加するように `kernel_spec` を変更する方法があります。[こちらをご覧ください](https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997)。


**Docker ユーザーの場合:** <a>TensorFlow のナイトリーを使用する Jupyter Notebook サーバー</a>の [Docker](https://docs.docker.com/install/) イメージを実行している場合は、ノートブックのポートだけでなく、TensorBoard のポートも公開する必要があります。次のコマンドでコンテナを実行します。

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter
```

上記の `-p 6006` は TensorBoard のデフォルトのポートです。これにより、1 つの TesorBoard インスタンスを実行するためのポートが割り当てられます。同時インスタンスを実行する場合は、さらにぽポートを割り当てる必要があります。また、`--bind_all` を `%tensorboard` に渡してコンテナの外部にポートを公開します。

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

TensorFlow、datetime、および os をインポートします。

In [ ]:
import tensorflow as tf
import datetime, os

## ノートブックにおける TensorBoard

[FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) データセットをダウンロードし、スケーリングします。

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


非常に単純なモデルを作成します。

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Keras と TensorBoard コールバックを使ってモデルをトレーニングします。

In [6]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 182us/sample - loss: 0.4976 - accuracy: 0.8204 - val_loss: 0.4143 - val_accuracy: 0.8538
Epoch 2/5
60000/60000 [==============================] - 10s 174us/sample - loss: 0.3845 - accuracy: 0.8588 - val_loss: 0.3855 - val_accuracy: 0.8626
Epoch 3/5
60000/60000 [==============================] - 10s 175us/sample - loss: 0.3513 - accuracy: 0.8705 - val_loss: 0.3740 - val_accuracy: 0.8607
Epoch 4/5
60000/60000 [==============================] - 11s 177us/sample - loss: 0.3287 - accuracy: 0.8793 - val_loss: 0.3596 - val_accuracy: 0.8719
Epoch 5/5
60000/60000 [==============================] - 11s 178us/sample - loss: 0.3153 - accuracy: 0.8825 - val_loss: 0.3360 - val_accuracy: 0.8782


[magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html) を使って、ノートブック内で TensorBoard を起動します。

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

スカラー、グラフ、ヒストグラムなどのダッシュボードを表示できるようになりました。一部のダッシュボード（プロファイルプラグインなど）はまだ Colab では使用できません。

`%tensorboard` マジックのフォーマットは、TensorBoard コマンドライン呼び出しとまったく同じですが、先頭に `%` 記号が付きます。

トレーニング前に TensorBoard を起動すると、その進捗状況を監視することもできます。

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

同じコマンドを発行すると、同じ TensorBoard バックエンドを再利用できます。異なる logs ディレクトリが選択されている場合、新しい TensorBoard インスタンスが開きます。ポートは自動的に管理されます。

新しいモデルのトレーニングを開始すると、TensorBoard が 30 秒ごとに自動更新されます。または、右上にあるボタンを使って再読み込みすることもできます。

In [9]:
train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 184us/sample - loss: 0.4968 - accuracy: 0.8223 - val_loss: 0.4216 - val_accuracy: 0.8481
Epoch 2/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3847 - accuracy: 0.8587 - val_loss: 0.4056 - val_accuracy: 0.8545
Epoch 3/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3495 - accuracy: 0.8727 - val_loss: 0.3600 - val_accuracy: 0.8700
Epoch 4/5
60000/60000 [==============================] - 11s 179us/sample - loss: 0.3282 - accuracy: 0.8795 - val_loss: 0.3636 - val_accuracy: 0.8694
Epoch 5/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3115 - accuracy: 0.8839 - val_loss: 0.3438 - val_accuracy: 0.8764


`tensorboard.notebook` API を使用すると、もう少し制御できるようになります。

In [10]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:00:54 ago; pid 265)


In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->